# Mortgage Characteristics in Qualified Opportunity Zones
_The focus of this map is to gather insight on the state of homeownership within and around areas with significant amounts of QOZs._ 

## [Link to map](https://arcg.is/1vaOXy)
Before these analyses, something of note I found was that on the QOZ FAQ the IRS specified that one does not have to live within the community a QOZ falls within to be taken advantage of. It is pertinent to see if current homeowners also benefit from being within tax incentivized areas or if they may contribute to an environment where homeowners' properities are being poached by outside enitities through the guise of economic development. 

### Pre-analysis Initialization

In [60]:
import arcpy
import os

# set the mypath as working folder
mypath = "E:\QOZ_Mortgages (work from school desktop)"

# name the qualified opportunity zone gdb
gdb = "QOZ_Mortgages.gdb"

# set  default folder
default_folder = "E:\QOZ_Mortgages (work from school desktop)"

In [61]:
# set overwite to true so you can overwite existing files
arcpy.env.workspace = os.path.join(mypath, gdb)
arcpy.env.overwriteOutput = True

In [62]:
# double checking the environment
print(arcpy.env.workspace)

E:\QOZ_Mortgages (work from school desktop)\QOZ_Mortgages.gdb


In [63]:
# listing feature names
fcs = arcpy.ListFeatureClasses()
print(fcs)

['StateMortgage', 'State_Atlas', 'State_Mortgage_Join', 'QOZ.shp', 'Residential_Construction_Permits_by_County.shp']


### Getting Census Mortgage Data
The variables of interest are characteristics of the residences that are owned within and around QOZs. This will be particularly helpful in surmising the state of homeownership within areas with high concentrations of QOZs. I hope to gain insight on whether areas with a significant amount of mortgages more or less likely to be located within the tax incentivized areas. I would also like to see whether new-builds or older homes are more likely to be surrounding QOZs. The answers to these two questions could give valuable evidence to see if QOZs are implemented as intended or are being monopolized by entities outside of the communities they fall within. 

The variables that I will call from the Census API are:
- __(S2506_C01_001E)__: Amount of owner-occupied housing units 
- __(S2506_C01_009E)__: Value of owner-occupied housing units with a mortgage in median dollars 
- __(S2506_C01_010E)__: Owner-occupied housing units with a mortgage and either a second mortgage or home equity loan 
- __(S2506_C02_066E)__: Real estate taxes for homeowners with a mortgage 
- __(DP04_0017E thru DP04_0016E)__: Decades housing units were built (from 1939 or ealier to 2020 or later)

In [5]:
import pandas as pd
import geopandas
import requests
import urllib3
import zipfile
import arcpy
import os

In [6]:
# necessary libraries for using Census API
from census import Census

In [ ]:
# set API key
API_Key = "My_Key"
c = Census(API_Key)

In [80]:
# define the variables being requested
# S2506 (financial characteristics for units w mortgage)
vars_s2506 = [
    "S2506_C01_001E",  # units with a mortgage
    "S2506_C01_009E",  # median mortgage value
    "S2506_C01_010E",  # units with mortgage + 2nd mortgage/HELOC
    "S2506_C02_066E",  # median real estate taxes for homeowners w/ mortgage
]

# DP04 (year structure built)
vars_dp04 = [
    "DP04_0017E",  # 2020 or later
    "DP04_0018E",  # 2010–2019
    "DP04_0019E",  # 2000–2009
    "DP04_0020E",  # 1990–1999
    "DP04_0021E",  # 1980–1989
    "DP04_0022E",  # 1970–1979
    "DP04_0023E",  # 1960–1969
    "DP04_0024E",  # 1950–1959
    "DP04_0025E",  # 1940–1949
]

# tract-level geography for ALL counties in California (state 06)
geo_ca_tracts = {
    "for": "tract:*",
    "in": "state:06 county:*"
}

In [81]:
# explore metadata
url = "https://api.census.gov/data/2023/acs/acs5/variables.json"  ## variables in 2023 ACS 5 year estimates
response = requests.get(url)  ## pull the URL
variable_list = response.json()  ## create a list of variables

## print a random sample of variables
for i, (var, details) in enumerate(variable_list["variables"].items()):
    if i >= 20:
        break
    print(f"{var}: {details.get('label')}")

for: Census API FIPS 'for' clause
in: Census API FIPS 'in' clause
ucgid: Uniform Census Geography Identifier clause
B24022_060E: Estimate!!Total:!!Female:!!Service occupations:!!Food preparation and serving related occupations
B19001B_014E: Estimate!!Total:!!$100,000 to $124,999
B07007PR_019E: Estimate!!Total:!!Moved from different municipio:!!Foreign born:!!Naturalized U.S. citizen
B19101A_004E: Estimate!!Total:!!$15,000 to $19,999
B24022_061E: Estimate!!Total:!!Female:!!Service occupations:!!Building and grounds cleaning and maintenance occupations
B19001B_013E: Estimate!!Total:!!$75,000 to $99,999
B07007PR_018E: Estimate!!Total:!!Moved from different municipio:!!Foreign born:
B19101A_005E: Estimate!!Total:!!$20,000 to $24,999
B19001B_012E: Estimate!!Total:!!$60,000 to $74,999
B24022_062E: Estimate!!Total:!!Female:!!Service occupations:!!Personal care and service occupations
B01001B_029E: Estimate!!Total:!!Female:!!65 to 74 years
B20005A_021E: Estimate!!Total:!!Male:!!Worked full-tim

In [82]:
# pull s2506 data
data_s = c.acs5st.get(
    vars_s2506 + ["NAME"],   # add NAME so you know what each row is
    geo=geo_ca_tracts,
    year=2023
)
df_s = pd.DataFrame(data_s)

# pulling dp04 data
data_dp = c.acs5dp.get(
    vars_dp04 + ["NAME"],
    geo=geo_ca_tracts,
    year=2023
)
df_dp = pd.DataFrame(data_dp)
df_dp

,DP04_0017E,DP04_0018E,DP04_0019E,DP04_0020E,DP04_0021E,DP04_0022E,DP04_0023E,DP04_0024E,DP04_0025E,NAME,state,county,tract
0,27.0,49.0,122.0,983.0,44.0,40.0,32.0,50.0,33.0,Census Tract 4001; Alameda County; California,06,001,400100
1,9.0,47.0,12.0,11.0,16.0,13.0,48.0,30.0,102.0,Census Tract 4002; Alameda County; California,06,001,400200
2,52.0,180.0,10.0,200.0,100.0,255.0,298.0,206.0,253.0,Census Tract 4003; Alameda County; California,06,001,400300
3,0.0,31.0,10.0,39.0,0.0,152.0,92.0,113.0,134.0,Census Tract 4004; Alameda County; California,06,001,400400
4,0.0,54.0,26.0,16.0,169.0,142.0,30.0,107.0,197.0,Census Tract 4005; Alameda County; California,06,001,400500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9124,0.0,301.0,71.0,58.0,74.0,19.0,55.0,39.0,17.0,Census Tract 409.02; Yuba County; California,06,115,040902
9125,8.0,85.0,271.0,266.0,357.0,258.0,162.0,47.0,16.0,Census Tract 410.01; Yuba County; California,06,115,041001
9126,12.0,130.0,345.0,387.0,395.0,179.0,103.0,64.0,0.0,Census Tract 410.02; Yuba County; California,06,115,041002
9127,0.0,24.0,170.0,242.0,252.0,428.0,29.0,154.0,33.0,Census Tract 411.01; Yuba County; California,06,115,041101


In [83]:
# merging the datasets
# they share these geo columns; NAME should also match
merge_cols = ["state", "county", "tract", "NAME"]
df = df_s.merge(df_dp, on=merge_cols, how="inner")
df

,S2506_C01_001E,S2506_C01_009E,S2506_C01_010E,S2506_C02_066E,NAME,state,county,tract,DP04_0017E,DP04_0018E,DP04_0019E,DP04_0020E,DP04_0021E,DP04_0022E,DP04_0023E,DP04_0024E,DP04_0025E
0,717.0,1751500.0,101.0,10001.0,Census Tract 4001; Alameda County; California,06,001,400100,27.0,49.0,122.0,983.0,44.0,40.0,32.0,50.0,33.0
1,329.0,1944700.0,59.0,10001.0,Census Tract 4002; Alameda County; California,06,001,400200,9.0,47.0,12.0,11.0,16.0,13.0,48.0,30.0,102.0
2,779.0,2000001.0,108.0,10001.0,Census Tract 4003; Alameda County; California,06,001,400300,52.0,180.0,10.0,200.0,100.0,255.0,298.0,206.0,253.0
3,528.0,1484400.0,62.0,10001.0,Census Tract 4004; Alameda County; California,06,001,400400,0.0,31.0,10.0,39.0,0.0,152.0,92.0,113.0,134.0
4,648.0,1444400.0,55.0,10001.0,Census Tract 4005; Alameda County; California,06,001,400500,0.0,54.0,26.0,16.0,169.0,142.0,30.0,107.0,197.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9124,12.0,-666666666.0,5.0,-666666666.0,Census Tract 409.02; Yuba County; California,06,115,040902,0.0,301.0,71.0,58.0,74.0,19.0,55.0,39.0,17.0
9125,777.0,480500.0,247.0,2783.0,Census Tract 410.01; Yuba County; California,06,115,041001,8.0,85.0,271.0,266.0,357.0,258.0,162.0,47.0,16.0
9126,690.0,658000.0,27.0,5470.0,Census Tract 410.02; Yuba County; California,06,115,041002,12.0,130.0,345.0,387.0,395.0,179.0,103.0,64.0,0.0
9127,371.0,362500.0,19.0,1665.0,Census Tract 411.01; Yuba County; California,06,115,041101,0.0,24.0,170.0,242.0,252.0,428.0,29.0,154.0,33.0


In [84]:
# some quick cleanup with pandas
##rename variables
df = df.rename(columns={
    "S2506_C01_001E": "units_w_mortgage",
    "S2506_C01_009E": "median_mortgage_value",
    "S2506_C01_010E": "units_w_mortgage_and_sec",
    "S2506_C02_066E": "median_re_taxes_mortgage",
    "DP04_0017E": "yrbuilt_2020_plus",
    "DP04_0018E": "yrbuilt_2010_2019",
    "DP04_0019E": "yrbuilt_2000_2009",
    "DP04_0020E": "yrbuilt_1990_1999",
    "DP04_0021E": "yrbuilt_1980_1989",
    "DP04_0022E": "yrbuilt_1970_1979",
    "DP04_0023E": "yrbuilt_1960_1969",
    "DP04_0024E": "yrbuilt_1950_1959",
    "DP04_0025E": "yrbuilt_1940_1949",
})

df.head()

,units_w_mortgage,median_mortgage_value,units_w_mortgage_and_sec,median_re_taxes_mortgage,NAME,state,county,tract,yrbuilt_2020_plus,yrbuilt_2010_2019,yrbuilt_2000_2009,yrbuilt_1990_1999,yrbuilt_1980_1989,yrbuilt_1970_1979,yrbuilt_1960_1969,yrbuilt_1950_1959,yrbuilt_1940_1949
0,717.0,1751500.0,101.0,10001.0,Census Tract 4001; Alameda County; California,06,001,400100,27.0,49.0,122.0,983.0,44.0,40.0,32.0,50.0,33.0
1,329.0,1944700.0,59.0,10001.0,Census Tract 4002; Alameda County; California,06,001,400200,9.0,47.0,12.0,11.0,16.0,13.0,48.0,30.0,102.0
2,779.0,2000001.0,108.0,10001.0,Census Tract 4003; Alameda County; California,06,001,400300,52.0,180.0,10.0,200.0,100.0,255.0,298.0,206.0,253.0
3,528.0,1484400.0,62.0,10001.0,Census Tract 4004; Alameda County; California,06,001,400400,0.0,31.0,10.0,39.0,0.0,152.0,92.0,113.0,134.0
4,648.0,1444400.0,55.0,10001.0,Census Tract 4005; Alameda County; California,06,001,400500,0.0,54.0,26.0,16.0,169.0,142.0,30.0,107.0,197.0


In [86]:
# create GOEID
df["GEOID"] = df["state"] + df["county"] + df["tract"]
df.head()

,units_w_mortgage,median_mortgage_value,units_w_mortgage_and_sec,median_re_taxes_mortgage,NAME,state,county,tract,yrbuilt_2020_plus,yrbuilt_2010_2019,yrbuilt_2000_2009,yrbuilt_1990_1999,yrbuilt_1980_1989,yrbuilt_1970_1979,yrbuilt_1960_1969,yrbuilt_1950_1959,yrbuilt_1940_1949,GEOID
0,717.0,1751500.0,101.0,10001.0,Census Tract 4001; Alameda County; California,06,001,400100,27.0,49.0,122.0,983.0,44.0,40.0,32.0,50.0,33.0,06001400100
1,329.0,1944700.0,59.0,10001.0,Census Tract 4002; Alameda County; California,06,001,400200,9.0,47.0,12.0,11.0,16.0,13.0,48.0,30.0,102.0,06001400200
2,779.0,2000001.0,108.0,10001.0,Census Tract 4003; Alameda County; California,06,001,400300,52.0,180.0,10.0,200.0,100.0,255.0,298.0,206.0,253.0,06001400300
3,528.0,1484400.0,62.0,10001.0,Census Tract 4004; Alameda County; California,06,001,400400,0.0,31.0,10.0,39.0,0.0,152.0,92.0,113.0,134.0,06001400400
4,648.0,1444400.0,55.0,10001.0,Census Tract 4005; Alameda County; California,06,001,400500,0.0,54.0,26.0,16.0,169.0,142.0,30.0,107.0,197.0,06001400500


## Adding boundary layers
Connecting the variable data to TIGER file to ultimately get it as a feature class in the geodatabase. 

In [87]:
# basic set up for loading the file
url = "https://www2.census.gov/geo/tiger/TIGER2023/TRACT/tl_2023_06_tract.zip"
filename = "tl_2023_06_tract.zip"

zip_path = os.path.join(default_folder, filename)

print(f"Download will come from {url}")
print(f"File will save to {zip_path}")

Download will come from https://www2.census.gov/geo/tiger/TIGER2023/TRACT/tl_2023_06_tract.zip
File will save to E:\QOZ_Mortgages (work from school desktop)\tl_2023_06_tract.zip


In [88]:
print("ZIP path is:", zip_path)
print("Exists?", os.path.exists(zip_path))

ZIP path is: E:\QOZ_Mortgages (work from school desktop)\tl_2023_06_tract.zip
Exists? True


In [89]:
# Unzip the shapefile
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(default_folder)
print(f"Unzipped {zip_path}")

# path to the extracted shapefile
shp_name = "tl_2023_06_tract.shp"
shp_path = os.path.join(default_folder, shp_name)

# load shapefile and check projection
boundaries = geopandas.read_file(shp_path)
print(boundaries.head())
print(boundaries.crs)

Unzipped E:\QOZ_Mortgages (work from school desktop)\tl_2023_06_tract.zip
  STATEFP COUNTYFP TRACTCE        GEOID               GEOIDFQ     NAME  \
0      06      001  442700  06001442700  1400000US06001442700     4427   
1      06      001  442800  06001442800  1400000US06001442800     4428   
2      06      037  204920  06037204920  1400000US06037204920  2049.20   
3      06      037  205110  06037205110  1400000US06037205110  2051.10   
4      06      037  320101  06037320101  1400000US06037320101  3201.01   

               NAMELSAD  MTFCC FUNCSTAT    ALAND  AWATER     INTPTLAT  \
0     Census Tract 4427  G5020        S  1234016       0  +37.5371513   
1     Census Tract 4428  G5020        S  1278646       0  +37.5293619   
2  Census Tract 2049.20  G5020        S   909972       0  +34.0175004   
3  Census Tract 2051.10  G5020        S   286962       0  +34.0245059   
4  Census Tract 3201.01  G5020        S   680504       0  +34.2992784   

       INTPTLON                           

In [91]:
# join on GEOID (or whatever your join field is)
merged = boundaries.merge(df, on="GEOID", how="left")
merged

,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME_x,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,tract,yrbuilt_2020_plus,yrbuilt_2010_2019,yrbuilt_2000_2009,yrbuilt_1990_1999,yrbuilt_1980_1989,yrbuilt_1970_1979,yrbuilt_1960_1969,yrbuilt_1950_1959,yrbuilt_1940_1949
0,06,001,442700,06001442700,1400000US06001442700,4427,Census Tract 4427,G5020,S,1234016,...,442700,0.0,0.0,8.0,44.0,136.0,120.0,462.0,311.0,0.0
1,06,001,442800,06001442800,1400000US06001442800,4428,Census Tract 4428,G5020,S,1278646,...,442800,0.0,0.0,0.0,7.0,19.0,213.0,649.0,56.0,0.0
2,06,037,204920,06037204920,1400000US06037204920,2049.20,Census Tract 2049.20,G5020,S,909972,...,204920,0.0,39.0,0.0,0.0,57.0,22.0,65.0,71.0,62.0
3,06,037,205110,06037205110,1400000US06037205110,2051.10,Census Tract 2051.10,G5020,S,286962,...,205110,0.0,0.0,0.0,0.0,14.0,63.0,42.0,154.0,201.0
4,06,037,320101,06037320101,1400000US06037320101,3201.01,Census Tract 3201.01,G5020,S,680504,...,320101,0.0,70.0,0.0,36.0,180.0,136.0,83.0,345.0,69.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9124,06,059,001303,06059001303,1400000US06059001303,13.03,Census Tract 13.03,G5020,S,1179647,...,001303,0.0,10.0,86.0,61.0,310.0,963.0,229.0,486.0,11.0
9125,06,059,001304,06059001304,1400000US06059001304,13.04,Census Tract 13.04,G5020,S,1252716,...,001304,20.0,114.0,37.0,40.0,166.0,306.0,41.0,205.0,76.0
9126,06,059,001401,06059001401,1400000US06059001401,14.01,Census Tract 14.01,G5020,S,1592982,...,001401,0.0,110.0,29.0,68.0,60.0,333.0,442.0,562.0,26.0
9127,06,013,367200,06013367200,1400000US06013367200,3672,Census Tract 3672,G5020,S,1322255,...,367200,7.0,124.0,408.0,0.0,64.0,298.0,217.0,274.0,433.0


In [ ]:
# save the new spatial layer as a shapefile
out_shp = os.path.join(default_folder, "merged_data.shp")
merged.to_file(out_shp)
print(f"Saved shapefile to {out_shp}")

# import shapefile into your geodatabase
arcpy.FeatureClassToFeatureClass_conversion(out_shp, gdb, "data")
print("Imported to geodatabase.")

## Statistical Analyses

I'm going to do the following statistical tests:
1. Global Moran's I to measure spatial autocorrolation
2. Getis-Ord Gi hotspot analysis to map clusters
3. Local Moran's I to interpret cluster types


#### _Global Moran's I Summary_

I applied Global Moran’s I to test whether the distribution of units with mortgages across census tracts exhibited a random or spatially structured pattern. The resulting Moran’s I value of _0.356_ indicates strong positive spatial autocorrelation, suggesting that high-mortgage tracts tend to cluster near other high mortgage tracts, while low values cluster together as well.

The extremely high z-score _(72.32)_ and p-value _(< 0.000001)_ confirm that this clustering is not due to random chance. Because the data exhibit significant spatial dependence, subsequent local spatial analyses such as Hot Spot Analysis and Cluster/Outlier Analysis are justified.


In [ ]:
# global moran's i
arcpy.stats.SpatialAutocorrelation(
    Input_Feature_Class="Mortgage Data",
    Input_Field="units_w_mo",
    Generate_Report=None,
    Conceptualization_of_Spatial_Relationships="K_NEAREST_NEIGHBORS",
    Distance_Method="EUCLIDEAN_DISTANCE",
    Standardization="ROW",
    Distance_Band_or_Threshold_Distance=None,
    Weights_Matrix_File=None,
    number_of_neighbors=8
)

#### _Getis-Ord Gi Summary_

The deep red areas that are largely in suburban zones such as the Inland Empire, Santa Clarita, Simi Valley, and much of Contra Costa and Alameda Counties indicate high concentrations of mortgage-holding households clustered together. These regions reflect places with strong homeownership markets, newer housing stock, and historically higher rates of mortgage uptake. The blue areas indicate cold spots, signaling neighborhoods with significantly lower levels of mortgaged homeownership. Cold spots appear notably central Los Angeles, Long Beach, Oakland, and parts of San Francisco. 

These tend to overlap with dense urban cores, areas with higher renter populations, affordability barriers, and zones experiencing slower residential investment. When viewed alongside the Qualified Opportunity Zones, the analysis suggests that QOZs often align with mortgage cold spots, reinforcing the idea that these federally targeted neighborhoods are systematically disinvested and structurally disconnected from the high ownership clusters that dominate surrounding suburban regions. This spatial contrast makes visible the uneven geography of homeownership and further underscores where investment and potential displacement pressures may concentrate over time.


In [ ]:
# hotspot analysis with getis-ord gi
arcpy.stats.HotSpots(
    Input_Feature_Class="Mortgage Data",
    Input_Field="units_w_mo",
    Output_Feature_Class=r"MortgageData_HotSpots",
    Conceptualization_of_Spatial_Relationships="K_NEAREST_NEIGHBORS",
    Distance_Method="EUCLIDEAN_DISTANCE",
    Standardization="ROW",
    Distance_Band_or_Threshold_Distance=None,
    Self_Potential_Field=None,
    Weights_Matrix_File=None,
    Apply_False_Discovery_Rate__FDR__Correction="NO_FDR",
    number_of_neighbors=8
)

#### _Local Moran's I Summary_
The Local Moran’s I analysis reveals a more nuanced view of mortgage ownership across the Los Angeles and Bay Area regions. 

- __High–High__ clusters (in dark red) identify neighborhoods where high levels of mortgaged homeownership are surrounded by similarly high values. These typically emerge in wealthier suburban zones such as Walnut Creek, San Ramon, Simi Valley, and pockets of the Inland Empire and North Bay. These areas reflect stable and investment-heavy housing markets where homeownership is the norm. 

- __Low–Low__ clusters (dark blue) highlight areas where low levels of mortgage ownership are surrounded by similarly low values. Such as central Los Angeles, Inglewood, Compton, Richmond, and parts of San Francisco and Oakland These patterns align closely with renter-dominated neighborhoods, historic disinvestment, and housing markets characterized by lower access to traditional mortgage financing.

- __High–Low__ (red squares) reveal isolated pockets of high mortgage ownership embedded within broader low mortgage areas, often marking sites of active redevelopment or recent suburban spillover. 

- __Low–High__ (blue squares) signal communities with disproportionately low mortgage rates adjacent to high-ownership zones, suggesting potential structural barriers or early signals of displacement pressure. Taken together, the  results show that mortgage ownership does not simply rise and fall regionally. The fact that many Qualified Opportunity Zones align with Low–Low clusters or Low–High outliers reinforces the idea that these zones are embedded within systematically undercapitalized housing markets, spatially disconnected from the higher ownership clusters that dominate adjacent suburban landscapes.

In [ ]:
# local moran's i
arcpy.stats.ClustersOutliers(
    Input_Feature_Class="Mortgage Data",
    Input_Field="units_w__1",
    Output_Feature_Class="MortgageData_ClustersOutliers",
    Conceptualization_of_Spatial_Relationships="K_NEAREST_NEIGHBORS",
    Distance_Method="EUCLIDEAN_DISTANCE",
    Standardization="ROW",
    Distance_Band_or_Threshold_Distance=None,
    Weights_Matrix_File=None,
    Apply_False_Discovery_Rate__FDR__Correction="NO_FDR",
    Number_of_Permutations=999,
    number_of_neighbors=8
)

## Results and Reflection

Across all three spatial analyses (Global Moran’s I, Getis-Ord Gi, and Local Moran’s I), the results show a clear and uneven geography of mortgage ownership in California. Mortgage holding households cluster strongly in suburban, higher income areas, while urban cores and historically disinvested neighborhoods form equally distinct clusters of low mortgage ownership. These low-ownership clusters align closely with many Qualified Opportunity Zones. This suggests that QOZs are overwhelmingly located in areas where mortgage access, wealth building, and long term residential stability have been limited for decades.

The hotspot analysis reinforces this pattern. High mortgage hot spots occur in stable, investment-heavy suburban markets that fall outside QOZ boundaries. Mortgage cold spots appear in renter dominated and lower income areas, which often map directly onto QOZ-designated tracts. The Local Moran’s I results add nuance by showing not only low ownership clusters but also outliers. Many QOZs sit within Low–Low clusters where weak homeownership is widespread, indicating long term structural barriers. But some appear as Low–High outliers, where neighborhoods with very low mortgage ownership sit next to strong ownership markets. These edge conditions often represent early stages of market pressure and are associated with redevelopment, speculation, and shifting property values.

Taken together, these spatial patterns raise important questions about the intended versus actual impacts of QOZs. While the program does target economically vulnerable neighborhoods, the location of QOZs within low ownership areas and adjacent to high-value suburban markets suggests a greater risk of fueling gentrification rather than supporting community based stability. Instead of reinforcing local economic stability, QOZ incentives may make these neighborhoods more attractive to outside investors, accelerating turnover and displacement pressures. In short, the spatial evidence points to QOZs functioning less as tools for stabilizing distressed communities and more as mechanisms that may amplify existing inequalities and encourage investment forward transformation.